In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d

In [ ]:
def lorentz(lhs,sigma,rval,bval):
    y1, y2, y3 = lhs[0], lhs[1], lhs[2]
    rhs = np.zeros(3, dtype=np.float64)
    rhs[0] = sigma*(y2-y1)
    rhs[1] = rval*y1-y2-y1*y3
    rhs[2] = -bval*y3 + y1*y2
    return rhs

# 4th order Runge-Kutta timestepper
def rk4(x0,f,dt):
    k1 = dt*f(x0)
    k2 = dt*f(x0 + k1/2.)
    k3 = dt*f(x0 + k2/2.)
    k4 = dt*f(x0 + k3)
    return x0 + (k1 + 2.*k2 + 2.*k3 + k4)/6.

# Time stepping scheme for solving x' = f(x) for t0<=t<=tf with time step dt. 
def timestepper(x0,t0,tf,dt,f):
    ndim = np.size(x0)
    nsteps = np.int((tf-t0)/dt)
    solpath = np.zeros((ndim,nsteps+1),dtype=np.float64)
    solpath[:,0] = x0
    for jj in range(nsteps):
        solpath[:,jj+1] = rk4(solpath[:,jj],f,dt)
    return solpath

In [ ]:
def dmd_cmp(gtot, thrshhld, window, ndsets):
    nrws, nclmns = gtot.shape
    gm = np.zeros((nrws, ndsets*(window-1)), dtype=np.float64)
    gp = np.zeros((nrws, ndsets*(window-1)), dtype=np.float64)
    # Perform DMD method.  Note, we need to be careful about how we break the concantenated Hankel matrix apart.  
    for ll in range(ndsets):
        gm[:, ll*(window-1):(ll+1)*(window-1)] = gtot[:, ll*window:(ll+1)*window-1]
        gp[:, ll*(window-1):(ll+1)*(window-1)] = gtot[:, 1+ll*window:(ll+1)*window]

    u, s, vh  = np.linalg.svd(gm, full_matrices=False)
    sm = np.max(s)
    indskp = np.log10(s/sm) > -thrshhld
    sr = s[indskp]
    ur = u[:, indskp]
    v = np.conj(vh.T)
    vr = v[:, indskp]
    kmat = gp @ vr @ np.diag(1. / sr) @ np.conj(ur.T)
    evls, evcs = np.linalg.eig(kmat)
    phim = (np.linalg.solve(evcs, gm))
    return evls, phim, evcs

def hankel_matrix(tseries,window):
    NT = np.size(tseries)
    nclmns = NT-(window-1)
    hmat = np.zeros((window,nclmns),dtype=np.float64)
    for jj in range(nclmns):
        hmat[:,jj] = tseries[jj:(jj+window)]
    sclfac = np.linalg.norm(hmat[:,-1])
    return [hmat, sclfac]

def hankel_dmd(rawdata, obs, window, thrshhld):
    NT = np.shape(rawdata[0])[1]
    nclmns = NT-(window-1)
    nobs = len(obs)
    ndsets = len(rawdata)
    
    hankel_mats = np.zeros((nclmns*nobs,window*ndsets),dtype=np.float64)
    for ll in range(ndsets):
        for jj in range(nobs):
            tseries = obs[jj](rawdata[ll])
            hmat, sclfac = hankel_matrix(tseries,window)
            if jj == 0:
                usclfac = sclfac     
            hankel_mats[jj*nclmns:(jj+1)*nclmns,ll*window:(ll+1)*window] = usclfac/sclfac * hmat.T   
    print("shape of hankel_mats: ", hankel_mats.shape)
    return dmd_cmp(hankel_mats, thrshhld, window, ndsets)

In [ ]:
def path_reconstruction(evls, phim, window, initconds):
    numics, dim = initconds.shape
    nobsvs, ntstps = phim.shape
    phimat = np.zeros((nobsvs, numics),dtype=np.complex128)
    kmat = np.zeros((nobsvs,dim),dtype=np.complex128)
    recon = np.zeros((dim, ntstps),dtype=np.float64)
    
    for jj in range(numics):
        phimat[:,jj] = phim[:,jj*(window-1)]
    
    u, s, vh = np.linalg.svd(phimat.T, full_matrices=False)
    initconds = np.diag(1./s) @ np.conj(u.T) @ initconds
    for ll in range(dim):
        kmat[:,ll] = np.conj(vh.T)@initconds[:,ll]
        print(np.linalg.norm(vh@kmat[:,ll] - initconds[:,ll])/np.linalg.norm(initconds[:,ll]))
        recon[ll,:] = np.real( kmat[:,ll].T @ phim )
    return recon

In [ ]:
def lorenz63_hankel(wndw, thrshhld=15, tf=20, t0=0, dt=.003, numIC=10, sigma=10., bval=8./3., rval=28.):
    window = int((tf-t0)/dt - wndw)
    x0 = np.array([5., 6., 7.])
    initconds = np.zeros((numIC,3))
    fhandle = lambda x: lorentz(x,sigma,rval,bval)
    obs1 = lambda x: x[0]
    obs = [obs1]
    rawdata = [None]*numIC

    for i in range(0,numIC):
        initconds[i,:] = x0 + np.random.randn(3)
        rawdata[i] = timestepper(initconds[i,:], t0, tf, dt, fhandle)
    
    evls, phim, kvecs = hankel_dmd(rawdata, obs, window, thrshhld)
    recon = path_reconstruction(evls, phim, window, initconds)
    return recon

In [ ]:
for wndw in range(1,51):
    recon = lorenz63_hankel(wndw, thrshhld=15)
    fig = plt.figure(figsize=(10,7))
    ax = plt.axes(projection="3d")
    ax.plot3D(recon[0,:], recon[1,:], recon[2,:])
    plt.title(f"Lorenz63: tau={wndw}, thrshhld=10")
    #plt.savefig(f"wndw1_50/thresh_15/lorenz63_hankel{wndw}.png")
    #plt.close(fig)